# **Team Assignment 2 - Predicting ROI**

## *IMPORTANT*: This is an outline of what needs to be discussed. Please fill in where you see fit and feel free to add more. If you don't understand something, I can explain. Please not the other files and folders in this project as they probably will anwser a lot of your questions. Also, you may have to play around with the paths since I use abosulte paths for everything which is relative to my VM. Overall, try to add as much detail as you can.

## 1. Project Overview
Start with an introduction that outlines the project's objectives, the significance of predicting ROI, and a brief overview of the approach taken. This section sets the stage for what the notebook will cover.

## 2. Data Overview
Before diving into the code, provide an overview of the data used for this project. Mention:

The source of the data.
Key features/variables and their relevance to the project.
Any preprocessing done outside of the notebook (e.g., data cleaning, feature selection) and the rationale behind these choices.


## 3. Computational Environment
Explain that the computational work, especially model training, was performed using UNC's Longleaf services due to the high computational requirements. Offer insight into why local machines were not suitable and how Longleaf services benefited the project.

## 4. Data Import and Initial Setup
Document how to set up the environment within the notebook, including importing necessary libraries and loading the dataset. If the data was preprocessed or features were selected outside the notebook, include code or pseudo-code snippets to show how this was done, or describe the process if code cannot be shared. Discuss why we used a virtual environment. Discuss project struture.

### 4.1 Setting the Correct Path

In [ ]:
import sys
from pathlib import Path
# This should be the path to the directory containing 'notebooks' and 'utils'
project_root = Path.cwd().parent  # If your notebook is directly inside the 'notebooks' directory
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

In [ ]:
# Check that you are in the right directory
%pwd

In [ ]:
%load_ext autoreload
%autoreload 2

### 4.2 Importing Needed Libraries

In [ ]:
import utils.utility as utility
import imputers.earliest_cr_line_d as ecld
import imputers.numeric_data as nd
# We first import a number of libraries that we will be using in today's class
import pandas as pd
import numpy as np

# Plotting packages we'll use
import matplotlib.pyplot as plt
import seaborn as sns

# Rather than importing the whole sklearn library, we will import only certain modules
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from joblib import load


### 4.3 Importing Accepted Loan Data

In [ ]:
path_to_accepted = '../data/LendingClub/accepted_2007_to_2018Q4.csv'

### 4.4 Read Accpeted Data into a Dataframe

In [ ]:
df_accepted = pd.read_csv(path_to_accepted)

In [ ]:
# Check rows and columns
print(df_accepted.shape)
df_accepted.head()

## 5. Primary Data Inspection and Feature Selection

### 5.1 Define a List of Features Available for us at Loan's Origin

In [10]:
classifier_features = ["acc_now_delinq", "acc_open_past_24mths", "addr_state","all_util",
    "annual_inc","annual_inc_joint","application_type","avg_cur_bal","bc_open_to_buy",
    "bc_util","chargeoff_within_12_mths","collections_12_mths_ex_med","delinq_2yrs",
    "dti","dti_joint","earliest_cr_line","emp_length","fico_range_high","fico_range_low",
    "funded_amnt","funded_amnt_inv","grade","home_ownership","il_util","initial_list_status",
    "inq_fi","inq_last_12m","inq_last_6mths","installment","int_rate","issue_d",
    "loan_amnt","max_bal_bc","mort_acc","mths_since_last_delinq","mths_since_last_major_derog",
    "mths_since_last_record","mths_since_rcnt_il","mths_since_recent_bc","mths_since_recent_inq",
    "num_accts_ever_120_pd","num_actv_bc_tl","num_actv_rev_tl","num_bc_sats","num_bc_tl","num_il_tl",
    "num_op_rev_tl","num_rev_accts","num_rev_tl_bal_gt_0","num_sats","num_tl_120dpd_2m","num_tl_30dpd",
    "num_tl_90g_dpd_24m","num_tl_op_past_12m","open_acc","open_acc_6m",
    "open_il_12m", "open_il_24m","open_act_il","open_rv_12m","open_rv_24m","pct_tl_nvr_dlq",
    "percent_bc_gt_75","pub_rec","pub_rec_bankruptcies","purpose","revol_bal","revol_util",
    "sub_grade","tax_liens","term","tot_coll_amt","tot_cur_bal","tot_hi_cred_lim","total_acc",
    "total_bal_ex_mort","total_bal_il","total_bc_limit","total_cu_tl","total_il_high_credit_limit",
    "verification_status","verified_status_joint"]


### 5.3 Drop Useless Columns

In [11]:
# Get the column names based on their positions
columns_to_drop = ['id', 'member_id', 'url', 'zip_code']

# Drop irrelevant columns
df_accepted = df_accepted.drop(columns_to_drop, axis=1)

### 5.4 Examine Features (Discuss Implications)

In [ ]:
df_accepted.info(verbose=True, show_counts=True)

In [ ]:
df_accepted.describe()

### 5.5 Dropping Columns with a High Null Count

In [12]:
# Drop where the null count is greater than or equal to 50% of samples.
df_accepted, null_columns = utility.drop_null_columns(df_accepted)
classifier_features = [item for item in classifier_features if item not in null_columns]

In [ ]:
# View null columns
print(null_columns)

In [ ]:
# View df's columns
print(list(df_accepted.columns))

## 6. Handling Missing Data

### 6.1 Imputing Dates

In [ ]:
df_accepted.info(verbose=True, show_counts=True)

In [13]:
# Get a list of all columns that could be dates (object type)
potential_date_columns = utility.get_object_columns(df_accepted)

In [ ]:
# Display each object with its value count
value_counts = utility.display_value_counts(df_accepted, potential_date_columns)
value_counts

In [15]:
# Drop all rows where issue_d is null
df_accepted = df_accepted.dropna(subset=['issue_d'])

In [16]:
# Type cast all date columns to date objects
date_columns = ['issue_d', 'earliest_cr_line']
df_accepted = utility.to_datetime(df=df_accepted, columns=date_columns)

#### 6.1.1 Create 'fico_descriptor' Column

In [17]:
df_accepted = utility.get_fico_descriptor(df=df_accepted)
df_accepted['fico_descriptor']

0               Good
1               Good
2               Good
3          Very Good
4               Good
             ...    
2260694         Fair
2260695         Good
2260696         Good
2260697         Fair
2260698         Fair
Name: fico_descriptor, Length: 2260668, dtype: object

In [18]:
# Impute missing dates
ecld_imputer = ecld.EarliestCRLineDateImputer()
ecld_imputer.fit(X=df_accepted)
df_accepted = ecld_imputer.transform(X=df_accepted)

### 6.2 Imputing Categorical Data

In [19]:
# Binned data
df_accepted = df_accepted[(df_accepted.emp_length != '10+ years') &
                          (df_accepted.emp_length != '< 1 year')]

# Make the rest categorical 
category_columns = utility.get_object_columns(df=df_accepted)
result_list = [item for item in category_columns if item not in date_columns]
df_accepted = utility.to_categorical(df=df_accepted, columns=result_list)

In [20]:
# Impute categorical data using mode
imputer = SimpleImputer(strategy='most_frequent')
df_accepted[category_columns] = imputer.fit_transform(df_accepted[category_columns])

In [ ]:
df_accepted.info(verbose=True, show_counts=True)

### 6.3 Imputing Numerical Data

In [21]:
numerical_columns = utility.get_numerical_columns(df_accepted)
columns_for_grouping_and_median = numerical_columns + ['fico_descriptor']
imputer = nd.NumericDataImputer(group_column='fico_descriptor')
imputer.fit(df_accepted[columns_for_grouping_and_median])
df_accepted[columns_for_grouping_and_median] = imputer.transform(df_accepted[columns_for_grouping_and_median])

/nas/longleaf/home/aryonna/488-team-assignment-2/imputers/numeric_data.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_transformed[column].fillna(self.medians_[column], inplace=True)


In [ ]:
df_accepted.info(verbose=True, show_counts=True)

## 7. Fixing Skewness

### 7.1 Examine data's distribution

In [ ]:
pd.set_option('display.max_columns', None)  # None means unlimited
df_accepted.describe()

In [22]:
# Get the columns with high skew values as these need to be fixed later
skewed_columns = utility.get_high_skewed_columns(df=df_accepted)

In [23]:
skewed_df = df_accepted[skewed_columns]
skewed_df.describe()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,last_fico_range_high,il_util
count,1.322675e+06,1.322675e+06,1.322675e+06,1.322675e+06,1.322675e+06,1.322675e+06
mean,1.440358e+04,1.439811e+04,1.437863e+04,1.314650e+01,6.859285e+02,7.156951e+01
std,8.946151e+03,8.944043e+03,8.947832e+03,4.824584e+00,7.380668e+01,1.740894e+01
min,5.000000e+02,5.000000e+02,0.000000e+00,5.310000e+00,0.000000e+00,0.000000e+00
25%,7.675000e+03,7.675000e+03,7.575000e+03,9.670000e+00,6.540000e+02,6.800000e+01
50%,1.200000e+04,1.200000e+04,1.200000e+04,1.269000e+01,6.990000e+02,7.300000e+01
75%,2.000000e+04,2.000000e+04,2.000000e+04,1.599000e+01,7.340000e+02,7.700000e+01
max,4.000000e+04,4.000000e+04,4.000000e+04,3.099000e+01,8.500000e+02,5.580000e+02


In [ ]:
# Visualize skewed features
utility.visualize_numerical_variables(skewed_df)

In [24]:
# Fix skewnes with log
log_df = utility.fix_skewed_features(log_df=df_accepted, skewed_features=skewed_columns)

/nas/longleaf/home/aryonna/488-team-assignment-2/utils/utility.py:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  log_df[f + '_log']=np.log1p(log_df[f])
/nas/longleaf/home/aryonna/488-team-assignment-2/utils/utility.py:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  log_df[f + '_log']=np.log1p(log_df[f])
/nas/longleaf/home/aryonna/488-team-assignment-2/utils/utility.py:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Conside

In [ ]:
log_df.info(verbose=True, show_counts=True)

## 8. Drop outliers

In [25]:
log_df = utility.remove_outliers(df=log_df)

## 9. Standardize Features

In [26]:
scaler = StandardScaler()
numeric_columns = utility.get_numerical_columns(df=log_df)
log_df = utility.scale_numeric(log_df, numeric_columns, scaler)

In [ ]:
log_df.info(verbose=True, show_counts=True)

In [27]:
# Save a copy
log_df_preFE = log_df.copy(deep=True)

## 10. Feature Engineer

In [28]:
# Credit history
log_df['credit_history_length'] = (log_df['issue_d'] - log_df['earliest_cr_line']).dt.days / 365.25

# PTI ratio
log_df['PTI_ratio'] = log_df['installment'] / (log_df['annual_inc']/12)

# Monthly disposable income
log_df['mnthly_disposable_income'] = (log_df['annual_inc'] / 12) - log_df['installment']

# Investor confidence
log_df['investor_confidence'] = log_df['funded_amnt_inv_log'] / log_df['loan_amnt_log']

# Utilization Ratios
log_df['avg_utilization_ratio'] = (log_df['revol_util'] + log_df['bc_util']) / 2

# Historical Delinquencies
log_df['delinquency_score'] = log_df['acc_now_delinq'] + log_df['delinq_2yrs'] + log_df['num_accts_ever_120_pd']

# Credit Inquiry Impact
log_df['credit_inquiry_impact'] = (log_df['inq_last_12m'] * 0.5) + (log_df['inq_last_6mths'] * 0.75) + log_df['inq_fi']

/tmp/ipykernel_3739205/2696554352.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  log_df['credit_history_length'] = (log_df['issue_d'] - log_df['earliest_cr_line']).dt.days / 365.25
/tmp/ipykernel_3739205/2696554352.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  log_df['PTI_ratio'] = log_df['installment'] / (log_df['annual_inc']/12)
/tmp/ipykernel_3739205/2696554352.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Con

In [ ]:
log_df.info(verbose=True, show_counts=True)

### 10.1 Creating Target Variable

In [29]:
# Calculate ROI
log_df = utility.calculate_roi(df=log_df)

/nas/longleaf/home/aryonna/488-team-assignment-2/utils/utility.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['total_received'] = df['total_pymnt'] + df['recoveries'] - df['collection_recovery_fee']
/nas/longleaf/home/aryonna/488-team-assignment-2/utils/utility.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['ROI'] = ((df['total_received'] - df['funded_amnt_log']) / df['funded_amnt_log'].replace(0, np.nan)) * 100


In [30]:
# Calculate ROI descriptor (High, Medium, Low) based on ROI
log_df = utility.get_roi_descriptor(df=log_df)

/nas/longleaf/home/aryonna/488-team-assignment-2/utils/utility.py:177: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['ROI_descriptor'] = df['ROI'].apply(categorize_roi)


In [31]:
# Edit classifier features list - can only include data that is available at the time the loan is drafted. Also removed any 
# features that were used to FE other ones to reduce dimensionality 
columns_to_drop = ['fico_range_high', 'fico_range_low', 'funded_amnt', 'funded_amnt_inv', 'il_util', 'int_rate', 'loan_amnt', 'verified_status_joint']
classifier_features = [item for item in classifier_features if item not in columns_to_drop]
classifier_features += ['ROI_descriptor']
classifier_features += ['loan_amnt_log', 'funded_amnt_log', 'funded_amnt_inv_log', 'int_rate_log', 'last_fico_range_high_log', 'il_util_log']

In [32]:
columns_to_drop = ['installment', 'annual_inc', 'funded_amnt_inv_log', 'loan_amnt_log',
                   'revol_util', 'bc_util', 'acc_now_delinq', 'delinq_2yrs',
                   'num_accts_ever_120_pd', 'inq_last_12m', 'inq_last_6mths', 'inq_fi', 'ROI'] + date_columns


classifier_features = [item for item in classifier_features if item not in columns_to_drop]

log_df = log_df[classifier_features]

In [ ]:
log_df.info(verbose=True)

## 11. Encoding Categorical Variables

In [33]:
category_columns = utility.get_object_columns(df=log_df)
log_df = utility.to_categorical(df=log_df, columns=category_columns)

/nas/longleaf/home/aryonna/488-team-assignment-2/utils/utility.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].astype('category')
/nas/longleaf/home/aryonna/488-team-assignment-2/utils/utility.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].astype('category')
/nas/longleaf/home/aryonna/488-team-assignment-2/utils/utility.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [ ]:
log_df.info(verbose=True, show_counts=True)

In [34]:
# Run once - must remove ROI_descriptor because this is our target 
category_columns = utility.get_category_columns(df=log_df)
category_columns.remove('ROI_descriptor')
print(category_columns)

['addr_state', 'application_type', 'emp_length', 'grade', 'home_ownership', 'initial_list_status', 'purpose', 'sub_grade', 'term', 'verification_status']


In [35]:
# One hot encoding on all (except ROI_descriptor)
log_df = utility.one_hot_encode(df=log_df, columns_to_encode=category_columns)

In [ ]:
log_df.info(verbose=True, show_counts=True)

In [36]:
# Save targets
y = log_df['ROI_descriptor'].values

In [37]:
# Check to see if number of y's = len of log_df
print(len(y))

700923


## 12. Training

In [ ]:
X = log_df.drop(columns='ROI_descriptor')

### 12.1 Explain that X and y are Saved for Cross Validation and Good Practice

In [ ]:
# (DO ONCE: ALREADY DONE) Save so we can use it to cross validate later
#dump(X, '/nas/longleaf/home/aryonna/488-team-assignment-2/data/raw_data/all_data/features.joblib')

# Save labels
#dump(y, '/nas/longleaf/home/aryonna/488-team-assignment-2/data/raw_data/all_data/target.joblib')

# EXPLANATION: This code saves X and y to a .joblib file for a model to use later once trained
# with a script. I'm just leaving this here so you can explain how/ where we got the data from. 
# This should not be ran. Simply here demo purposes.

### 12.2 Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

### 12.3 Explain that X_train and y_train are Saved for Training the Model

In [ ]:
# (DO ONCE: ALREADY DONE) SAVE FOR TRAINING THE MODEL
#dump(X_train, '/nas/longleaf/home/aryonna/488-team-assignment-2/data/train_data/all_data/features.joblib')

# Save labels
#dump(y_train, '/nas/longleaf/home/aryonna/488-team-assignment-2/data/train_data/all_data/target.joblib')

# EXPLANATION: This code saves X_train and y_train to a .joblib file for a model to use later to
# train a model with a script. I'm just leaving this here so you can explain how/ where we got the data from. 
# This should not be ran. Simply here demo purposes.

In [ ]:
# After training, these features were deemed the most important based on "mean" meaning their 
# imporatnce was over the average importance. These were generated from the rfc.py training script
most_important_features = ["dti", "max_bal_bc", "revol_bal", "tot_hi_cred_lim", "total_bal_ex_mort", 
                           "total_bal_il", "total_bc_limit", "funded_amnt_log", "int_rate_log", 
                           "last_fico_range_high_log"]

### 12.4 Reduce Data Based on Fearture Importance

In [ ]:
reduced_data = X[most_important_features]

In [ ]:
# SAVE RAW REDUCED DATA FOR CROSS VALIDATION
#dump(reduced_data, '/nas/longleaf/home/aryonna/488-team-assignment-2/data/raw_data/reduced_data/features.joblib')

# Save labels
#dump(y, '/nas/longleaf/home/aryonna/488-team-assignment-2/data/raw_data/reduced_data/target.joblib')

# EXPLANATION: This code saves X and y to a .joblib file for cross validation on the important features
# I'm just leaving this here so you can explain how/ where we got the data from. 
# This should not be ran. Simply here demo purposes.

In [ ]:
# Loading the model that was generated in the rfc.py script
rfc = load('/nas/longleaf/home/aryonna/488-team-assignment-2/models/rfc_reduced_data.joblib')
y_pred = rfc.predict(X_test[most_important_features])

In [ ]:
# Check the accuracy on the test set
accuracy_rf = rfc.score(X_test[most_important_features], y_test)
print(f"Accuracy of Random Forest: {accuracy_rf}")


## 13. Cross-Validation

In [ ]:
# Define the file path from which to read the results
cross_val_results_path = '/nas/longleaf/home/aryonna/488-team-assignment-2/cross-val-results/rfc_reduced_results.txt'  # Update this to your actual file path

# Open the file in read mode ('r') and print its contents
with open(cross_val_results_path, 'r') as file:
    results = file.read()

# Print the results
print(results)

## 14. Possible Optimizations

## 15. Conclusions and Findings